# Create a grade sheet
This notebook provides sample [EDSL](https://docs.expectedparrot.com/) code for designing a gradesheet as an EDSL survey conducted for a set of material to be graded.

EDSL is an open-source library for simulating surveys and experiments with AI. Please see our [documentation page](https://docs.expectedparrot.com/) for tips and tutorials on getting started.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/expectedparrot/edsl/blob/main/docs/notebooks/grading_experiment.ipynb)

In [1]:
# ! pip install edsl

In [2]:
from edsl.questions import QuestionYesNo, QuestionList
from edsl import Scenario, Agent, Survey, Model

Here we define the notion of a "requirement" that is worth some number of points:

In [3]:
class Requirement:
    common_header = """Consider this idea for a {{ type }} business: {{ idea }}."""

    def __init__(self, requirement, grading_instructions, question_name, points):
        self.requirement = requirement
        self.grading_instructions = grading_instructions
        self.question_name = question_name
        self.points = points

    @property
    def question_text(self):
        return (
            self.common_header
            + "\n"
            + self.requirement
            + "\n"
            + self.grading_instructions
            + "\n"
            + "Does this idea satisfy the requirement?"
        )

    def question(self):
        return QuestionYesNo(
            question_text=self.question_text, question_name=self.question_name
        )


r1 = Requirement(
    requirement="The idea is clear and well-written.",
    grading_instructions=".",
    question_name="clear_idea",
    points=10,
)

r2 = Requirement(
    requirement="The idea has obvious user benefits.",
    grading_instructions="Describe the benefits of different user types.",
    question_name="user_benefits",
    points=10,
)

r3 = Requirement(
    requirement="The idea will attract a large number of new users.",
    grading_instructions="Explain why.",
    question_name="new_users",
    points=10,
)

requirements = [r1, r2, r3]
survey = Survey([r.question() for r in requirements])

Here we create a gradesheet for the requirements:

In [4]:
class Gradesheet:
    def __init__(self, result, requirements):
        self.result = result
        self.requirements = requirements
        self.points = 0
        self._problems = []

    def grade(self):
        for requirement in self.requirements:
            answer = self.result.sub_dicts["answer"].get(requirement.question_name)
            if answer == "Yes":
                self.points += requirement.points
            # if answer == "No":
            explain = self.result.sub_dicts["answer"].get(
                requirement.question_name + "_comment"
            )
            self._problems.append(
                (requirement.question_name, str(requirement.points), explain)
            )

        return self.points

    def problems(self):
        return self._problems

    def sheet(self):
        from rich.console import Console
        from rich.table import Table

        console = Console()
        table = Table(show_header=True, header_style="bold magenta")

        # Add columns to the table
        table.add_column("Requirement")
        table.add_column("Points")
        table.add_column("Issue")

        for problem in self._problems:
            table.add_row(*problem)
            table.add_row("", "", "")  # Add an empty row as a separator between items

        console.print(table)

Here we generate some ideas for demonstration:

In [5]:
def get_ideas(num_ideas, type=""):
    q = QuestionList(
        question_name="ideas",
        question_text=f"Draft a list of { num_ideas } business ideas for a { type } business.",
    )
    ideas = q.run(progress_bar=True).select("ideas").to_list()[0]
    return ideas


ideas = get_ideas(5, "startup")
ideas

Output()

['Sustainable Packaging Solutions',
 'Telehealth Services',
 'AI-Powered Personal Finance App',
 'Eco-Friendly Fashion Brand',
 'Smart Home Automation Systems']

Here we create some personas for agents that will grade the ideas, runs the survey with the questions parameterized with the ideas, and then shows the results in gradesheet tables:

In [ ]:
personas = [
    "You are a business analyst.",
    "You are a venture capital investor.",
    "You are an employee at a startup.",
]

agents = [Agent(traits={"persona": p}) for p in personas]

scenarios = [Scenario({"type": "startup", "idea": i}) for i in ideas]

results = survey.by(scenarios).by(agents).run(progress_bar=True)

for i in ideas:
    for p in personas:
        print("Idea: ", i)
        print("Persona: ", p)
        g = Gradesheet(
            results.filter(
                "scenario.idea == '" + i + "' and agent.persona == '" + p + "'"
            )[0],
            requirements,
        )
        print("Total points earned: ", g.grade())
        g.sheet()

Output()